<a href="https://colab.research.google.com/github/jthetzel/colab/blob/master/yolo_opencv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!kill -9 -1

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
··········


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
!wget https://pjreddie.com/media/files/yolov3.weights -P drive/collab/yolo-coco

In [0]:
!pip install imutils pafy youtube-dl

    100% |████████████████████████████████| 1.8MB 3.1MB/s 


In [0]:
!pip install --upgrade youtube_dl

Requirement already up-to-date: youtube_dl in /usr/local/lib/python3.6/dist-packages (2019.1.10)


In [0]:
import pafy
url = "https://www.youtube.com/watch?v=0LHxvxdRnYc"
vPafy = pafy.new(url)
play = vPafy.getbest(preftype="mp4")
video_path = 'drive/collab/video.mp4'
play.download(filepath=video_path)

'drive/collab/video.mp4'

In [0]:
video_path = 'drive/collab/video.mp4'
import imutils
import cv2
vs = cv2.VideoCapture(video_path)
# try to determine the total number of frames in the video file
try:
        prop = cv2.cv.CV_CAP_PROP_FRAME_COUNT if imutils.is_cv2() \
                else cv2.CAP_PROP_FRAME_COUNT
        total = int(vs.get(prop))
        print("[INFO] {} total frames in video".format(total))

# an error occurred while trying to determine the total
# number of frames in the video file
except:
        print("[INFO] could not determine # of frames in video")
        print("[INFO] no approx. completion time can be provided")
        total = -1


In [0]:
play.get_filesize()

11878809

In [0]:
import numpy as np
import imutils
import time
import cv2
import os

yolo_path = "drive/collab/yolo-coco"
output_path = "drive/collab/yolo-output/test.avi"

# load the COCO class labels our YOLO model was trained on
labelsPath = os.path.sep.join([yolo_path, "coco.names"])
LABELS = open(labelsPath).read().strip().split("\n")

# initialize a list of colors to represent each possible class label
np.random.seed(42)
COLORS = np.random.randint(0, 255, size=(len(LABELS), 3),
        dtype="uint8")

# derive the paths to the YOLO weights and model configuration
weightsPath = os.path.sep.join([yolo_path, "yolov3.weights"])
configPath = os.path.sep.join([yolo_path, "yolov3.cfg"])

# load our YOLO object detector trained on COCO dataset (80 classes)
# and determine only the *output* layer names that we need from YOLO
print("[INFO] loading YOLO from disk...")
net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)
ln = net.getLayerNames()
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]

# initialize the video stream, pointer to output video file, and
# frame dimensions
vs = cv2.VideoCapture(video_path)
writer = None
(W, H) = (None, None)

# try to determine the total number of frames in the video file
try:
        prop = cv2.cv.CV_CAP_PROP_FRAME_COUNT if imutils.is_cv2() \
                else cv2.CAP_PROP_FRAME_COUNT
        total = int(vs.get(prop))
        print("[INFO] {} total frames in video".format(total))

# an error occurred while trying to determine the total
# number of frames in the video file
except:
        print("[INFO] could not determine # of frames in video")
        print("[INFO] no approx. completion time can be provided")
        total = -1


[INFO] loading YOLO from disk...
[INFO] 3316 total frames in video


In [0]:
# loop over frames from the video file stream
progress = 0
while True:
        # read the next frame from the file
        (grabbed, frame) = vs.read()

        # if the frame was not grabbed, then we have reached the end
        # of the stream
        if not grabbed:
                print("NOT GRABED")
                break

        # if the frame dimensions are empty, grab them
        if W is None or H is None:
                (H, W) = frame.shape[:2]

        # construct a blob from the input frame and then perform a forward
        # pass of the YOLO object detector, giving us our bounding boxes
        # and associated probabilities
        blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416),
                swapRB=True, crop=False)
        net.setInput(blob)
        start = time.time()
        layerOutputs = net.forward(ln)
        end = time.time()

        # initialize our lists of detected bounding boxes, confidences,
        # and class IDs, respectively
        boxes = []
        confidences = []
        classIDs = []

        # loop over each of the layer outputs
        for output in layerOutputs:
                # loop over each of the detections
                for detection in output:
                        # extract the class ID and confidence (i.e., probability)
                        # of the current object detection
                        scores = detection[5:]
                        classID = np.argmax(scores)
                        confidence = scores[classID]

                        # filter out weak predictions by ensuring the detected
                        # probability is greater than the minimum probability
                        if confidence > 0.5:
                                # scale the bounding box coordinates back relative to
                                # the size of the image, keeping in mind that YOLO
                                # actually returns the center (x, y)-coordinates of
                                # the bounding box followed by the boxes' width and
                                # height
                                box = detection[0:4] * np.array([W, H, W, H])
                                (centerX, centerY, width, height) = box.astype("int")

                                # use the center (x, y)-coordinates to derive the top
                                # and and left corner of the bounding box
                                x = int(centerX - (width / 2))
                                y = int(centerY - (height / 2))

                                # update our list of bounding box coordinates,
                                # confidences, and class IDs
                                boxes.append([x, y, int(width), int(height)])
                                confidences.append(float(confidence))
                                classIDs.append(classID)

        # apply non-maxima suppression to suppress weak, overlapping
        # bounding boxes
        idxs = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.3)

        # ensure at least one detection exists
        if len(idxs) > 0:
                # loop over the indexes we are keeping
                for i in idxs.flatten():
                        # extract the bounding box coordinates
                        (x, y) = (boxes[i][0], boxes[i][1])
                        (w, h) = (boxes[i][2], boxes[i][3])

                        # draw a bounding box rectangle and label on the frame
                        color = [int(c) for c in COLORS[classIDs[i]]]
                        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
                        text = "{}: {:.4f}".format(LABELS[classIDs[i]],
                                confidences[i])
                        cv2.putText(frame, text, (x, y - 5),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        # check if the video writer is None
        if writer is None:
                # initialize our video writer
                fourcc = cv2.VideoWriter_fourcc(*"MJPG")
                writer = cv2.VideoWriter(output_path, fourcc, 30,
                        (frame.shape[1], frame.shape[0]), True)

                # some information on processing single frame
                if total > 0:
                        elap = (end - start)
                        print("[INFO] single frame took {:.4f} seconds".format(elap))
                        print("[INFO] estimated total time to finish: {:.4f}".format(
                                elap * total/60))

        # write the output frame to disk
        #cv2.imshow("Image", frame)
        #cv2.waitKey(1)
        progress += 1
        print(progress/ total *100)
        writer.write(frame)

# release the file pointers
print("[INFO] cleaning up...")
writer.release()
vs.release()

[INFO] single frame took 1.9977 seconds
[INFO] estimated total time to finish: 110.4061
0.030156815440289503
0.060313630880579006
0.09047044632086852
0.12062726176115801
0.15078407720144751
0.18094089264173704
0.21109770808202655
0.24125452352231602
0.27141133896260555
0.30156815440289503
0.33172496984318456
0.3618817852834741
0.39203860072376356
0.4221954161640531
0.4523522316043426
0.48250904704463204
0.5126658624849216
0.5428226779252111
0.5729794933655006
0.6031363088057901
0.6332931242460796
0.6634499396863691
0.6936067551266586
0.7237635705669482
0.7539203860072377
0.7840772014475271
0.8142340168878166
0.8443908323281062
0.8745476477683957
0.9047044632086853
0.9348612786489747
0.9650180940892641
0.9951749095295537
1.0253317249698433
1.0554885404101326
1.0856453558504222
1.1158021712907118
1.1459589867310012
1.1761158021712907
1.2062726176115801
1.2364294330518697
1.2665862484921593
1.2967430639324489
1.3268998793727382
1.3570566948130276
1.3872135102533172
1.4173703256936068
1.44